# ETL Lab

### Introduction

In this lab, we ask you to use the techniques learned in this section to work with an API of your choosing.  As important to getting to the correct code is to develop the proper procedure for getting there.  Just like in the preceding lessons we will following our procedures such as: 

1. Red, green, refactor
2. Move mess into an object
3. Make small methods by: 
    A. Commenting code
    B. Translating comments into methods
    
Along the way, we will arrive at our pattern of a *Client*, *Adapter*, and *Target*.

### Step 1.  Just get the data

The first step is to go from red to green.  That is, the code starts off with nothing working and our task is simply to get it working.  In this case, this means the following: 

1. Call an API of your choosing
2. Return a list of dictionaries and store as a variable named `entities`

In [8]:
import requests
response = requests.get("https://data.seattle.gov/resource/kzjm-xkqj.json")
entities = response.json()


In [9]:
type(entities)
# list

list

In [10]:
type(entities[0])
# dict

dict

In [11]:
entities[0]

{'address': '420 Vine St',
 'type': '4RED - 2 + 1 + 1',
 'datetime': '2022-09-13T16:31:00.000',
 'latitude': '47.617379',
 'longitude': '-122.34693',
 'report_location': {'type': 'Point', 'coordinates': [-122.34693, 47.617379]},
 'incident_number': 'F220111423',
 ':@computed_region_ru88_fbhk': '56',
 ':@computed_region_kuhn_3gp2': '7',
 ':@computed_region_q256_3sug': '19576'}

### Step 2. Change the dictionaries into objects

The next step is to change dictionaries received back from the API into objects.  We can break this down into a couple of steps.

1. Create the *target class*.  This is the class the dictionaries will be transformed into.  To do this, choose no more than five attributes to store in each instance.

In [30]:
class Calls911:
    def __init__(self, type, datetime, latitude, longitude):
        self.type = type
        self.datetime = datetime
        self.latitude = latitude
        self.longitude = longitude

Check your work by assigning an instance to the variable `target_instance`.

In [32]:
call911 = Calls911(type=entities[0]['type'], datetime=entities[0]['datetime'], latitude=entities[0]['latitude'], longitude=entities[0]['longitude'])
3 < len( call911.__dict__.keys()) < 5
# True 

True

1. Reject some of the data

We don't want to pass all of our data into our class.  So create a smaller dictionary of just the attributes we need.

In [33]:
attributes = ('type', 'datetime', 'latitude', 'longitude')
selected_attributes = dict((k, entities[0][k]) for k in attributes)

In [36]:
type(selected_attributes)
# dict

dict

In [35]:
len(selected_attributes.keys()) == len(call911.__dict__.keys())
# True

True

2. Coerce dictionaries into objects

A. To start, coerce just one dictionary into an object.

In [37]:
first_object = Calls911(**selected_attributes)
# change the above line to reference your target class

In [38]:
first_object.__dict__

{'type': '4RED - 2 + 1 + 1',
 'datetime': '2022-09-13T16:31:00.000',
 'latitude': '47.617379',
 'longitude': '-122.34693'}

In [39]:
list(first_object.__dict__.values()) == list(entities[0].values())
# True

False

B. Now that you have solved for one, solve for all.  Coerce all of the dictionaries into objects.  Assign the list of objects to a variable `targets`.

In [40]:

calls911 = [Calls911(**dict((k, e[k]) for k in attributes)) for e in entities if e.get('latitude') is not None]

In [41]:
len(calls911) == len(entities)
# True

False

### 3. Move the remaining code into an object 

At this point, we successfully have transformed a list of dictionaries from an API into a list of objects.  But we need to keep cleaning up our code.  To do this, look at the code outside of a class, and move it into a class, with a method name of run.

In [42]:
class Builder911calls:
    def run(self):
        import requests
        response = requests.get("https://data.seattle.gov/resource/kzjm-xkqj.json")
        entities = response.json()        
        attributes = ('type', 'datetime', 'latitude', 'longitude')
        calls911 = [Calls911(**dict((k, e[k]) for k in attributes)) for e in entities if e.get('latitude') is not None]
        return calls911

Let's marke sure that this works.

In [43]:
adapter = Builder911calls()
# change the above line to reference your adapter
results = adapter.run()

In [49]:
len(results) == len(entities)
# True

False

### 4. Make the methods smaller

Next, separate out the run method in the adapter class smaller.  Do this by first writing comments in the code, and then moving the code into separate methods.  Please leave the comments in your code.  Your methods should be no longer than five lines long, and there can only be a total of one `if else` statement or `loop` per method.  Having both an `if else` and a `loop` in any method is also too complicated -- don't do it.

In [45]:
class Builder911calls:
    def run(self):
        # get data
        import requests
        response = requests.get("https://data.seattle.gov/resource/kzjm-xkqj.json")
        entities = response.json()        
        # create objects
        attributes = ('type', 'datetime', 'latitude', 'longitude')
        calls911 = [Calls911(**dict((k, e[k]) for k in attributes)) for e in entities if e.get('latitude') is not None]
        return calls911

### 5. Create the client class

Next move calls to the API into their own separate class.  This way we can call the API but later to decide to coerce the data into different types of objects than we did above.

In [46]:
class Client911:
    URL = "https://data.seattle.gov/resource/kzjm-xkqj.json"
    def run(self):
        response = requests.get(self.URL)
        return response.json()   

Place the updated Adapter class below.  Check that it still works as it did before.

In [50]:
class Builder911callsRef:
    def run(self):
        self._request_api = Client911()
        self._calls_data = self._request_api.run()
        return self.calls_data_to_objects(self._calls_data)

    def calls_data_to_objects(self, entities):
        attributes = ('type', 'datetime', 'latitude', 'longitude')
        calls911 = [Calls911(**dict((k, e[k]) for k in attributes)) for e in entities if e.get('latitude') is not None]
        return calls911

In [51]:
refactored_adapter = Builder911callsRef()

len(refactored_adapter.run()) == len(adapter.run())

True

### Summary

Great job!  Hopefully, you saw how by building our code and then slowly refactoring our code, we can eventually get to some clean code.